In [181]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Chargement des données

In [182]:
url_links="/content/links.csv"
url_movies="/content/movies.csv"
url_ratings="/content/ratings.csv"
url_tags="/content/tags.csv"

In [183]:
# Lire les fichiers CSV
df_links = pd.read_csv(url_links)
df_movies = pd.read_csv(url_movies)
df_ratings = pd.read_csv(url_ratings)
df_tags = pd.read_csv(url_tags)

#Echantillonage des données

Nous allons prendre un echantillons stratifés des données afin de garder la distribution et prendre que 10% des données

In [184]:

from sklearn.model_selection import train_test_split
df_links = pd.DataFrame(df_links)

# Échantillonnage stratifié
links, _ = train_test_split(
    df_links,
    train_size=0.1,  # Garde 10% des données
    stratify=None,   # Pas de colonne catégorielle spécifique ici, peut être ajusté si nécessaire
    random_state=42  # Assure la reproductibilité
)

# Résultat
print("Données originales :")
print(df_links)

print("\nÉchantillon stratifié (10%) :")
print(links)

Données originales :
       movieId    imdbId     tmdbId
0            1    114709      862.0
1            2    113497     8844.0
2            3    113228    15602.0
3            4    114885    31357.0
4            5    113041    11862.0
...        ...       ...        ...
87580   292731  26812510  1032473.0
87581   292737  14907358   986674.0
87582   292753  12388280   948139.0
87583   292755     64027   182776.0
87584   292757  28995566  1174725.0

[87585 rows x 3 columns]

Échantillon stratifié (10%) :
       movieId   imdbId    tmdbId
4947      5052   268696  125717.0
74163   242112    26774  229401.0
43220   164566  1104688   53805.0
9967     33585   411705      27.0
65629   213081    77374  334680.0
...        ...      ...       ...
6265      6383   322259     584.0
54886   189071  4015500  369523.0
76820   254506   277013  104749.0
860        879   120004   11015.0
15795    83264  1610452   52696.0

[8758 rows x 3 columns]


In [187]:
df_movies = pd.DataFrame(df_movies)

# Échantillonnage stratifié
movies, _ = train_test_split(
    df_movies,
    train_size=0.1,  # Garde 10% des données
    stratify=None,   # Pas de colonne catégorielle spécifique ici, peut être ajusté si nécessaire
    random_state=42  # Assure la reproductibilité
)

# Résultat
print("Données originales :")
print(df_movies)

print("\nÉchantillon stratifié (10%) :")
print(movies)

Données originales :
       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
87580   292731           The Monroy Affaire (2022)   
87581   292737          Shelter in Solitude (2023)   
87582   292753                         Orca (2023)   
87583   292755              The Angry Breed (1968)   
87584   292757           Race to the Summit (2023)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...         

In [199]:
df_ratings = pd.DataFrame(df_ratings)

# Échantillonnage stratifié
ratings, _ = train_test_split(
    df_ratings,
    train_size=0.1,  # Garde 10% des données
    stratify=None,   # Pas de colonne catégorielle spécifique ici, peut être ajusté si nécessaire
    random_state=42  # Assure la reproductibilité
)

# Résultat
print("Données originales :")
print(df_ratings)

print("\nÉchantillon stratifié (10%) :")
print(ratings)

Données originales :
          userId  movieId  rating   timestamp
0              1       17     4.0   944249077
1              1       25     1.0   944250228
2              1       29     2.0   943230976
3              1       30     5.0   944249077
4              1       32     5.0   943228858
...          ...      ...     ...         ...
32000199  200948    79702     4.5  1294412589
32000200  200948    79796     1.0  1287216292
32000201  200948    80350     0.5  1294412671
32000202  200948    80463     3.5  1350423800
32000203  200948    87304     4.5  1350423523

[32000204 rows x 4 columns]

Échantillon stratifié (10%) :
          userId  movieId  rating   timestamp
13819805   86396     4911     3.0  1151067035
21336762  133475     4085     1.5  1219336025
17311034  108287     7156     4.0  1378236000
6441751    40195    48322     2.5  1171982968
5960822    37256     6297     4.0  1533004918
...          ...      ...     ...         ...
26858567  168782     8464     3.5  1112815546

In [189]:
df_tags = pd.DataFrame(df_tags)

# Échantillonnage stratifié
tags, _ = train_test_split(
    df_tags,
    train_size=0.1,  # Garde 10% des données
    stratify=None,   # Pas de colonne catégorielle spécifique ici, peut être ajusté si nécessaire
    random_state=42  # Assure la reproductibilité
)

# Résultat
print("Données originales :")
print(df_tags)

print("\nÉchantillon stratifié (10%) :")
print(tags)

Données originales :
         userId  movieId             tag   timestamp
0            22    26479     Kevin Kline  1583038886
1            22    79592        misogyny  1581476297
2            22   247150      acrophobia  1622483469
3            34     2174           music  1249808064
4            34     2174           weird  1249808102
...         ...      ...             ...         ...
2000067  162279    90645      Rafe Spall  1320817734
2000068  162279    91079   Anton Yelchin  1322337407
2000069  162279    91079  Felicity Jones  1322337400
2000070  162279    91658     Rooney Mara  1325828398
2000071  162279   100714     Julie Delpy  1373095449

[2000072 rows x 4 columns]

Échantillon stratifié (10%) :
         userId  movieId                       tag   timestamp
157326    23053   156706                 dirty cop  1632037016
767168    78213     4615                 pregnancy  1634701261
1433486   95129     1527        visually appealing  1614209979
1818226  144758     1892        

#Transformation de données

#Création des tables

In [190]:
ratings

,userId,movieId,rating,timestamp
13819805,86396,4911,3.0,1151067035
21336762,133475,4085,1.5,1219336025
17311034,108287,7156,4.0,1378236000
6441751,40195,48322,2.5,1171982968
5960822,37256,6297,4.0,1533004918
...,...,...,...,...
26858567,168782,8464,3.5,1112815546
23327850,146077,5139,0.5,1179184174
16094478,100887,224264,3.5,1683412792
23200604,145266,40617,3.0,1494796925


In [200]:


# Convertir les autres colonnes en types appropriés
ratings['userId'] = ratings['userId'].astype(int)
ratings['movieId'] = ratings['movieId'].astype(int)
ratings['rating'] = ratings['rating'].astype(int)
# Supprimer les lignes avec des valeurs NaN (causées par des valeurs non convertibles)
ratings = ratings.dropna(subset=['rating'])

# Vérification
print(ratings.dtypes)
print(ratings.head())


userId       int64
movieId      int64
rating       int64
timestamp    int64
dtype: object
          userId  movieId  rating   timestamp
13819805   86396     4911       3  1151067035
21336762  133475     4085       1  1219336025
17311034  108287     7156       4  1378236000
6441751    40195    48322       2  1171982968
5960822    37256     6297       4  1533004918


In [201]:
# Vérifier les doublons
duplicates = ratings[ratings.duplicated(subset=['userId', 'movieId'], keep=False)]
print(f"Doublons détectés : {len(duplicates)}")


Doublons détectés : 0


In [202]:

# Vérifier les valeurs non numériques dans la colonne rating
invalid_values = ratings[~ratings['rating'].apply(lambda x: isinstance(x, (int, float)))]

print(invalid_values)

Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []


In [203]:
# Convertir la colonne 'timestamp' en date
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Afficher le résultat pour vérifier
print(ratings.head())



# Afficher le DataFrame pour vérifier que la colonne a bien été supprimée
print(ratings.head())

          userId  movieId  rating           timestamp
13819805   86396     4911       3 2006-06-23 12:50:35
21336762  133475     4085       1 2008-08-21 16:27:05
17311034  108287     7156       4 2013-09-03 19:20:00
6441751    40195    48322       2 2007-02-20 14:49:28
5960822    37256     6297       4 2018-07-31 02:41:58
          userId  movieId  rating           timestamp
13819805   86396     4911       3 2006-06-23 12:50:35
21336762  133475     4085       1 2008-08-21 16:27:05
17311034  108287     7156       4 2013-09-03 19:20:00
6441751    40195    48322       2 2007-02-20 14:49:28
5960822    37256     6297       4 2018-07-31 02:41:58


In [204]:
ratings

,userId,movieId,rating,timestamp
13819805,86396,4911,3,2006-06-23 12:50:35
21336762,133475,4085,1,2008-08-21 16:27:05
17311034,108287,7156,4,2013-09-03 19:20:00
6441751,40195,48322,2,2007-02-20 14:49:28
5960822,37256,6297,4,2018-07-31 02:41:58
...,...,...,...,...
26858567,168782,8464,3,2005-04-06 19:25:46
23327850,146077,5139,0,2007-05-14 23:09:34
16094478,100887,224264,3,2023-05-06 22:39:52
23200604,145266,40617,3,2017-05-14 21:22:05


# teste


In [205]:
# Vérifier les valeurs invalides dans la colonne rating (qui doit être un FLOAT)
ratings['rating'] = pd.to_numeric(ratings['rating'], errors='coerce')

# Vérifier s'il y a des valeurs NaN dans 'rating'
invalid_ratings = ratings[ratings['rating'].isna()]
print("Valeurs invalides dans 'rating':")
print(invalid_ratings)

# Remplacer les NaN par une valeur par défaut (par exemple, 0) ou supprimer les lignes
ratings['rating'].fillna(0)  # Ou utilisez dropna() pour supprimer les lignes problématiques

# Vérifier les valeurs uniques dans les colonnes 'userId' et 'movieId' (doivent être des entiers)
ratings['userId'] = ratings['userId'].astype(int)
ratings['movieId'] = ratings['movieId'].astype(int)

# Vérifier les données après nettoyage
print(ratings.head())

Valeurs invalides dans 'rating':
Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []
          userId  movieId  rating           timestamp
13819805   86396     4911       3 2006-06-23 12:50:35
21336762  133475     4085       1 2008-08-21 16:27:05
17311034  108287     7156       4 2013-09-03 19:20:00
6441751    40195    48322       2 2007-02-20 14:49:28
5960822    37256     6297       4 2018-07-31 02:41:58


In [206]:
print(ratings.dtypes)

userId                int64
movieId               int64
rating                int64
timestamp    datetime64[ns]
dtype: object


* **Création de la table user**

In [207]:
merged = pd.merge(ratings, movies, on='movieId', how='inner')

# Regroupement par userId et création d'une colonne avec la liste des films
Utilisateur = merged.groupby('userId')['title'].agg(lambda x: list(x)).reset_index()

# Affichage de la table avec la liste des films par utilisateur
print(Utilisateur)

        userId                                              title
0            1           [Patton (1970), Crying Game, The (1992)]
1            3  [Batman Returns (1992), Armageddon (1998), Tee...
2            5                                [Waterworld (1995)]
3            6                   [Lara Croft: Tomb Raider (2001)]
4            7                [Die Hard: With a Vengeance (1995)]
...        ...                                                ...
114775  200943  [Marathon Man (1976), Star Wars: Episode I - T...
114776  200944       [Beowulf (2007), Million Dollar Baby (2004)]
114777  200945           [Gifted (2017), Crying Game, The (1992)]
114778  200947                                 [Inception (2010)]
114779  200948  [Phenomenon (1996), Terminator, The (1984), Ed...

[114780 rows x 2 columns]


**Vérifier si vraiment on a la liste des utilisateur avec tous les films visualiser**

In [208]:

# Fonction pour obtenir les données d'un utilisateur spécifique en fonction de son userId
def get_user_data_by_id(df, user_id):
    """
    Récupère toutes les données d'un utilisateur en fonction de son userId.

    :param df: DataFrame
    :param user_id: L'ID de l'utilisateur
    :return: Un DataFrame avec les données de l'utilisateur spécifié
    """
    user_data = df[df['userId'] == user_id]
    return user_data

# Exemple : Obtenir les données de l'utilisateur avec userId = 1
user_data = get_user_data_by_id(Utilisateur, 200948   )

# Afficher les données de l'utilisateur
print(user_data)

        userId                                              title
114779  200948  [Phenomenon (1996), Terminator, The (1984), Ed...


# Chargement des données (téléchargement)

In [209]:
ratings.to_csv('ratings_fact.csv', index=False)
movies.to_csv('table_movies.csv', index=False)
Utilisateur.to_csv('table_Utilisateur.csv', index=False)


print("Le fichier 'ratings.csv' a été téléchargé avec succès.")

Le fichier 'ratings.csv' a été téléchargé avec succès.
